In [5]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import zipfile
import os
import urllib.request

In [6]:
# Step 1: Download and extract the dataset
url = "https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip"
data_dir = "cats_and_dogs_filtered"
if not os.path.exists(data_dir):
    urllib.request.urlretrieve(url, "cats_and_dogs_filtered.zip")
    with zipfile.ZipFile("cats_and_dogs_filtered.zip", 'r') as zip_ref:
        zip_ref.extractall()

In [7]:
# Paths for training and validation data
train_dir = os.path.join(data_dir, "train")
validation_dir = os.path.join(data_dir, "validation")

In [8]:
# Step 2: Data Preprocessing
train_datagen = ImageDataGenerator(       #It preprocesses images and applies augmentation to enhance model generalization.
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)
validation_datagen = ImageDataGenerator(rescale=1./255)   #It normalizes pixel values to the range [0, 1], improving model performance.

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary'
)
validation_generator = validation_datagen.flow_from_directory(
    validation_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary'
)

Found 2000 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.


In [9]:
# Step 3: Load Pre-trained Model
base_model = MobileNetV2(weights='imagenet', include_top=False)       #It excludes the fully connected layers, allowing customization for the new task.

# Freeze the base model
base_model.trainable = False

<ipython-input-9-518d74350156>:2: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [96, 128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  base_model = MobileNetV2(weights='imagenet', include_top=False)


9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [10]:
# Step 4: Build the Model
model = Sequential([
    base_model,
    GlobalAveragePooling2D(),             #It reduces the feature map dimensions by averaging, making the model more efficient.
    Dropout(0.2),
    Dense(1, activation='sigmoid')        #For binary classification, sigmoid outputs probabilities for each class.
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [11]:
# Step 5: Train the Model
history = model.fit(
    train_generator,
    validation_data=validation_generator,
    epochs=5
)

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/5
63/63 ━━━━━━━━━━━━━━━━━━━━ 157s 2s/step - accuracy: 0.7884 - loss: 0.4411 - val_accuracy: 0.9790 - val_loss: 0.1028
Epoch 2/5
63/63 ━━━━━━━━━━━━━━━━━━━━ 137s 2s/step - accuracy: 0.9302 - loss: 0.1716 - val_accuracy: 0.9770 - val_loss: 0.0763
Epoch 3/5
63/63 ━━━━━━━━━━━━━━━━━━━━ 134s 2s/step - accuracy: 0.9431 - loss: 0.1408 - val_accuracy: 0.9750 - val_loss: 0.0792
Epoch 4/5
63/63 ━━━━━━━━━━━━━━━━━━━━ 135s 2s/step - accuracy: 0.9424 - loss: 0.1380 - val_accuracy: 0.9790 - val_loss: 0.0639
Epoch 5/5
63/63 ━━━━━━━━━━━━━━━━━━━━ 148s 2s/step - accuracy: 0.9591 - loss: 0.1175 - val_accuracy: 0.9800 - val_loss: 0.0649


In [12]:
# Step 6: Evaluate the Model
loss, accuracy = model.evaluate(validation_generator)
print(f"Validation Accuracy: {accuracy:.2f}")

32/32 ━━━━━━━━━━━━━━━━━━━━ 38s 1s/step - accuracy: 0.9813 - loss: 0.0612
Validation Accuracy: 0.98


In [13]:
# Save the model
model.save('cat_dog_recognition_model.h5')
print("Model saved successfully!")

Model saved successfully!


In [ ]:
#Classes Loaded: Successfully loaded 2000 training images and 1000 validation images, belonging to 2 classes (Cats and Dogs).
#MobileNetV2 Warning: The warning about input_shape is harmless since the default input shape (224, 224) was loaded correctly.
#Training Results:
#Achieved 98% validation accuracy after 5 epochs.
#Loss values decreased steadily, indicating good learning progress.
#Model Saved: The model was saved successfully in HDF5 format, but you might consider saving in the newer .keras format as suggested.

#conclusion:     Achieved High Accuracy: The model successfully achieved a validation accuracy of 98%, indicating excellent performance in distinguishing between cats and dogs.
                #Efficient Use of Transfer Learning: By leveraging the pre-trained MobileNetV2, the program demonstrated how transfer learning can save time and resources while maintaining high accuracy.
                 #Robust Pipeline: The inclusion of data augmentation and normalization ensured that the model generalized well to unseen data.
                 #Reusable Model: The trained model was saved, making it ready for future use without the need to retrain.

In [ ]:
#Transfer learning is a technique where a pre-trained model is adapted to solve a new problem, leveraging its existing knowledge.

#MobileNetV2 is lightweight, efficient, and optimized for mobile and edge devices while retaining high accuracy.

#advantages of transfer learning?
               #Faster training, reduced data requirements, and leveraging pre-trained features for better accuracy.

#What kind of problem is being solved in this program?
                    #Binary classification for recognizing images of cats and dogs.

In [ ]:
#What loss function is used here, and why?
       #Binary crossentropy, as it’s suitable for binary classification problems.
#What does the accuracy metric indicate during training?
       #It measures the proportion of correctly classified images.
#Why do we use validation data during training?
        #To monitor model performance and avoid overfitting.
#What does the training history (loss and accuracy trends) tell us?
         #It shows how well the model is learning and generalizing over epochs.

#Why might the validation accuracy differ from training accuracy?
         #Due to overfitting, data distribution differences, or noise in the validation set.

#How would you handle overfitting if it occurred?
         #By using techniques like dropout, more data augmentation, or reducing model complexity.

# evaluate() function do?
        #It computes the loss and accuracy on the given dataset.